****Get/scrape 2024 election results - no official statistical reports yet****

In [35]:
# Set loc and import pkgs

In [37]:
from selenium import webdriver
import time
import pandas as pd
import os
import io

In [39]:
pwd

'/Users/sunny/Library/CloudStorage/OneDrive-Personal/Documents/Python/Electoral maps project'

In [41]:
os.chdir("/Users/sunny/Library/CloudStorage/OneDrive-Personal/Documents/Python/Electoral maps project")

In [43]:
pwd

'/Users/sunny/Library/CloudStorage/OneDrive-Personal/Documents/Python/Electoral maps project'

**First attempt: listing all valid URLs**

In [ ]:
# Scrape 2024 election data
import requests

base_url = "https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS"

for state_code in range(1, 30):  # Assuming 35 states/UTs
    for constituency_number in range(1, 81):  # Assuming up to 99 constituencies per state
        # Construct the URL
        url = f"{base_url}{state_code:02}{constituency_number}.htm#"
        
        try:
            # Send a GET request
            response = requests.get(url, timeout=5)  # Timeout of 5 seconds
            if response.status_code == 200:
                print(f"URL exists: {url}")
            else:
                print(f"URL does not exist (status {response.status_code}): {url}")
        except requests.exceptions.RequestException as e:
            print(f"Error accessing {url}: {e}")


In [1]:
# Doesn't work
# Need Selenium

In [57]:
# Doesn't seem to be working in headless mode
from selenium.webdriver.common.by import By

# Path to your ChromeDriver
cService = webdriver.ChromeService(executable_path = '/usr/local/bin/chromedriver')

# Initialize Selenium WebDriver
driver = webdriver.Chrome(service = cService)


In [ ]:
# List to store valid URLs
valid_urls = []

# Loop through state codes and constituency numbers

# Base URL
base_url = "https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS"

for state_code in range(1, 30):  # 2 states -> assuming up to 30 state codes
    for constituency_number in range(1, 81):  # up to 80 constituencies per state
        # Construct the URL
        url = f"{base_url}{state_code:02}{constituency_number}.htm"

        try:
           # Open the URL in the browser
            driver.get(url)
            time.sleep(2)  # Allow time for the page to load

            # Check if the page contains valid content (adjust condition as needed)
            if "Election Commission of India" in driver.page_source:
                print(f"URL exists: {url}")
                valid_urls.append({"state_code": state_code, "constituency_number": constituency_number, "url": url})
            else:
                print(f"Invalid URL found, stopping search for state {state_code}: {url}")
                break  # Exit inner loop if invalid URL is found
        except Exception as e:
            print(f"Error accessing {url}: {e}")
            break  # Exit inner loop on error

# Loop through union territory codes and constituency numbers
# Base URL
base_url = "https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseU"

for ut_code in range(1, 20):  # Assuming up to 20 UT codes
    for constituency_number in range(1, 81):  # Assuming up to 80 constituencies
        # Construct the URL
        url = f"{base_url}{ut_code:02}{constituency_number}.htm"

        try:
            # Open the URL in the browser
            driver.get(url)
            time.sleep(2)  # Allow time for the page to load

            # Check if the page contains valid content (adjust condition as needed)
            if "Election Commission of India" in driver.page_source:
                print(f"URL exists: {url}")
                valid_urls.append({"state_code": ut_code, "constituency_number": constituency_number, "url": url})
            else:
                print(f"Invalid URL found, stopping search for UT {ut_code}: {url}")
                break  # Exit inner loop if invalid URL is found
        except Exception as e:
            print(f"Error accessing {url}: {e}")
            break  # Exit inner loop on error


# Close the browser
driver.quit()

# Save valid URLs to a CSV file
valid_urls_df = pd.DataFrame(valid_urls)
valid_urls_df.to_csv("valid_urls.csv", index=False)
print("Valid URLs saved to 'valid_urls.csv'")


**Second task: actually scrape data from all valid URLs**

Test one URL

In [ ]:
from selenium.webdriver.common.by import By

# Path to your ChromeDriver
cService = webdriver.ChromeService(executable_path = '/usr/local/bin/chromedriver')

# Initialize Selenium WebDriver
driver = webdriver.Chrome(service = cService)

# Test URL
test_url = "https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS1025.htm"

# Define the XPath of the table
table_xpath = '/html/body/main/div/div[3]'
header_xpath = '/html/body/main/div/div[1]/h2/span'

try:
    # Open the test URL in the browser
    driver.get(test_url)
    time.sleep(2)  # Allow time for the page to load

    # Scrape the table
    table_element = driver.find_element(By.XPATH, table_xpath)
    table_html = table_element.get_attribute('outerHTML')
    table_df = pd.read_html(io.StringIO(table_html))[0]

    # Scrape the constituency name
    header_element = driver.find_element(By.XPATH, header_xpath)
    constituency_name = header_element.text  # Extract the text content

    # Add the constituency name to the DataFrame
    table_df['Constituency'] = constituency_name

    # Print the DataFrame
    print(table_df)

except Exception as e:
    print(f"Error scraping the data: {e}")

finally:
    # Close the browser
    driver.quit()


Loop over all valid URLs

In [31]:
# Path to your ChromeDriver
cService = webdriver.ChromeService(executable_path = '/usr/local/bin/chromedriver')

# Initialize Selenium WebDriver
driver = webdriver.Chrome(service = cService)

# Load the valid URLs from the CSV
valid_urls_df = pd.read_csv("valid_urls.csv")

# Define the XPaths
table_xpath = '/html/body/main/div/div[3]'  # XPath for the table
header_xpath = '/html/body/main/div/div[1]/h2/span'  # XPath for the constituency name

# Define a list to store all constituency data
all_data = []

# Loop through all valid URLs
for _, row in valid_urls_df.iterrows():
    url = row['url']
    print(f"Scraping URL: {url}")

    try:
        # Open the URL in the browser
        driver.get(url)
        time.sleep(5)  # Allow time for the page to load

        # Scrape the table
        table_element = driver.find_element(By.XPATH, table_xpath)
        table_html = table_element.get_attribute('outerHTML')
        table_df = pd.read_html(io.StringIO(table_html))[0]

        # Scrape the constituency name
        header_element = driver.find_element(By.XPATH, header_xpath)
        constituency_name = header_element.text  # Extract the text content

        # Add the constituency name to the DataFrame
        table_df['Constituency'] = constituency_name

        # Add additional context from the URL (state and constituency numbers)
        table_df['State Code'] = row['state_code']
        table_df['Constituency Number'] = row['constituency_number']

        # Append the data to the list
        all_data.append(table_df)

    except Exception as e:
        print(f"Error scraping {url}: {e}")

# Close the browser
driver.quit()

# Combine all data into a single DataFrame
full_results_df = pd.concat(all_data, ignore_index=True)

# Save the results to a CSV file
full_results_df.to_csv("election_results.csv", index=False)

print("Scraping complete. Results saved to election_results.csv")


Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS011.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS012.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS013.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS014.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS015.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS016.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS017.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS018.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS019.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS0110.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS0111.htm
Scraping URL: https://results.eci.gov.in/PcResultGen

Append extra

In [61]:
# Path to your ChromeDriver
cService = webdriver.ChromeService(executable_path = '/usr/local/bin/chromedriver')

# Initialize Selenium WebDriver
driver = webdriver.Chrome(service = cService)

# Load the valid URLs from the CSV
valid_urls_df = pd.read_csv("valid_urls_extra.csv")

# Define the XPaths
table_xpath = '/html/body/main/div/div[3]'  # XPath for the table
header_xpath = '/html/body/main/div/div[1]/h2/span'  # XPath for the constituency name

# Define a list to store all constituency data
all_data = []

# Loop through all valid URLs
for _, row in valid_urls_df.iterrows():
    url = row['url']
    print(f"Scraping URL: {url}")

    try:
        # Open the URL in the browser
        driver.get(url)
        time.sleep(5)  # Allow time for the page to load

        # Scrape the table
        table_element = driver.find_element(By.XPATH, table_xpath)
        table_html = table_element.get_attribute('outerHTML')
        table_df = pd.read_html(io.StringIO(table_html))[0]

        # Scrape the constituency name
        header_element = driver.find_element(By.XPATH, header_xpath)
        constituency_name = header_element.text  # Extract the text content

        # Add the constituency name to the DataFrame
        table_df['Constituency'] = constituency_name

        # Add additional context from the URL (state and constituency numbers)
        table_df['State Code'] = row['state_code']
        table_df['Constituency Number'] = row['constituency_number']

        # Append the data to the list
        all_data.append(table_df)

    except Exception as e:
        print(f"Error scraping {url}: {e}")

# Close the browser
driver.quit()

# Combine all data into a single DataFrame
full_results_df = pd.concat(all_data, ignore_index=True)

# Save the results to a CSV file
full_results_df.to_csv("election_results_extra.csv", index=False)

print("Scraping complete. Results saved to election_results_extra.csv")


Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS251.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS252.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS253.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS254.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS255.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS256.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS257.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS258.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS259.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS2510.htm
Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS2511.htm
Scraping URL: https://results.eci.gov.in/PcResultGen

In [52]:
#NB:
#Scraping URL: https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS251.htm
#Error scraping https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS251.htm: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/main/div/div[3]"}